In [27]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [28]:
def canny(img):

    canny = cv.Canny(img, 50, 150)
    return canny

In [29]:
def region_of_interest(img):
    height = img.shape[0]
    poly = np.array([[(200,height), (1200, height), (790,450), (580, 450)]])
    mask = np.zeros_like(img)
    cv.fillPoly(mask, poly, 255)
    return mask

In [30]:
def display_lines(img, lines):
    line_image = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv.line(line_image, (x1,y1), (x2,y2), (0,255,0), 10)
    return line_image

In [45]:
def average_slope_intercept(lines):
    left_lines    = [] # (slope, intercept)
    left_weights  = [] # (length,)
    right_lines   = [] # (slope, intercept)
    right_weights = [] # (length,)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2==x1:
                continue # ignore a vertical line
            slope = (y2-y1)/(x2-x1)
            intercept = y1 - slope*x1
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)
            if slope < 0: # y is reversed in image
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
    
    # add more weight to longer lines    
    left_lane  = np.dot(left_weights,  left_lines) /np.sum(left_weights)  if len(left_weights) >0 else None
    right_lane = np.dot(right_weights, right_lines)/np.sum(right_weights) if len(right_weights)>0 else None
    
    return left_lane, right_lane # (slope, intercept), (slope, intercept)

In [46]:
def make_line_points(y1, y2, line,old_slope):
    if line is None:
        return "not"
    
    slope, intercept = line
    #print("slope = ", slope)
    #print("intercept = ", intercept)
    
    # make sure everything is integer as cv2.line requires it
    x1=int((y1 - intercept)/old_slope) if slope ==0 else int((y1 - intercept)/slope)
    x2= int((y2 - intercept)/old_slope) if slope ==0 else int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    
    return ((x1, y1), (x2, y2))

In [47]:
def lane_lines(image, lines,left_slope,right_slope):
    left_lane, right_lane = average_slope_intercept(lines)
    
    y1 = 680 # bottom of the image
    y2 = y1*0.725     # slightly lower than the middle

    left_line  = make_line_points(y1, y2, left_lane,left_slope)
    right_line = make_line_points(y1-5, y2, right_lane,right_slope)
    return left_line, right_line,left_lane,right_lane

    
def draw_lane_lines(image, lines, color=[255,0, 0], thickness=10):
    # make a separate image to draw lines and combine with the orignal later
    line_image = np.zeros_like(image)
    for line in lines:
        if line != "not":
            cv.line(line_image, *line,  color, thickness)
    # image1 * α + image2 * β + λ
    # image1 and image2 must be the same shape.
    return cv.addWeighted(image, 1.0, line_image, 0.8, 0.0)

In [48]:

def lane(left_line,right_line,img):
    poly = np.array([[(left_line[0][0],left_line[0][1]), (right_line[0][0], right_line[0][1]),  (right_line[1][0], right_line[1][1]),(left_line[1][0],left_line[1][1])]])
    mask = np.zeros_like(img)
    cv.fillPoly(mask, poly, color=(0, 255, 0))
    return mask

In [49]:
cap = cv.VideoCapture('project_video.mp4')
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")



left_line_old =((200,680),(790,450))
right_line_old=((1200,680),(580,450))
left_slope_old =100
right_slope_old=100

while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    # Display the resulting frame

    
    hls = cv.cvtColor(frame, cv.COLOR_BGR2HLS)    
    low_yellow = np.uint8([ 10,   0, 100])
    up_yellow = np.uint8([ 40, 255, 255])
    mask = cv.inRange(hls, low_yellow, up_yellow)
    gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    blur = cv.GaussianBlur(gray, (3,3), 0)
    interestt = blur & region_of_interest(blur)
    
    ret, thresh = cv.threshold(interestt, 210, 240, cv.THRESH_BINARY)
    
    new = mask | thresh
    
    interestt = new & region_of_interest(new)

    new_canny = cv.Canny(interestt, 50, 150)
    lines = cv.HoughLinesP(new_canny, 5, np.pi/180, 100, np.array([]), minLineLength=50, maxLineGap=200)
    
    left_line, right_line , left_slope, right_slope=lane_lines(frame, lines,left_slope_old,right_slope_old) 
    new_lines=left_line, right_line
    
    new_lines=list(new_lines)
    if  new_lines[0] == "not":
         new_lines[0]=left_line_old
    else:
        left_line_old= new_lines[0]

        
    if new_lines[1] == "not":
        new_lines[1]=right_line_old
    else:
        right_line_old=new_lines[1]
    
    if left_slope is not None:
        if left_slope[0] != 0:
            left_slope_old=left_slope[0]
    if right_slope is not None:
        if right_slope[0] !=0:
            right_slope_old=right_slope[0]
        
    
    lane_mask = lane( new_lines[0],new_lines[1],frame)

    x = draw_lane_lines(frame,new_lines)    
    lanes_img = cv.addWeighted(x, 1.0, lane_mask, 1.0, 0.0)
    cv.imshow('Frame',lanes_img)

    
        
    # Press Q on keyboard to  exit
    if cv.waitKey(25) & 0xFF == ord('q'):
      break

  else: 
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv.destroyAllWindows()